# Mummies

In [1]:
import networkx as nx
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 30)

## Loading data

In [3]:
mummies_df = pd.read_excel('data/Combined%20Mummy%20Database.xlsx', header=1)
mummies_df.drop('Unnamed: 4', inplace=True, axis=1)
mummies_df.drop('Unnamed: 25', inplace=True, axis=1)
mummies_df.fillna('', inplace=True)

In [4]:
mummies_df.shape

(960, 24)

In [5]:
mummies_df.head()

,Name,Impact Number,Institution,Contact person?,Designation,Animal,Human,Male,Female,Subadult,Adult,Lower Age,Upper Age,Lower Date,Upper Date,Dynasty,Period,Status,site,reference,xray?,CT?,comments,pathology
0,Male Mummy Kiev,NON,Pechersk Lavra,Dr. Mykola Teresenko,,,y,y,,,y,,,11th C BCE,10th C BCE,,,,,http://www.express.co.uk/news/world/806511/egy...,,,elite burial style,
1,Crocodile,NON,Pechersk Lavra,Dr. Mykola Teresenko,,y,,,,,,,,,,,,,,http://www.express.co.uk/news/world/806511/egy...,,,161cm,
2,Menat,NON,Pechersk Lavra,Dr. Mykola Teresenko,,,y,,y,,y,,,,,,,,,http://www.express.co.uk/news/world/806511/egy...,,,post-mortem decapitation,
3,Neswaiu,NON,Museum of Meditarranean and Near Eastern Antiq...,,,,y,,,,,50,60,3rd c BC,,,,,,http://3dprint.com/60734/3d-scanned-printed-mu...,,yes,also 3d printed,
4,Nesmen Khetamun,NON,Shrewsbury Museum & Art Gallery,,,,y,,,,,40,,600bc,,,,,,http://www.shropshirestar.com/news/2015/01/24/...,,,,


## Transforming data

### Unique name

In [6]:
counts = mummies_df.Name.value_counts()

In [7]:
for i, row in mummies_df.iterrows():
    if not row.Name:
        name = 'Unknown ---' + str(i)
    elif counts[row.Name] == 1:
        name = row.Name.strip()
    else:
        name = row.Name.strip() + ' ---' + str(i)
    mummies_df.loc[i, 'UNIQUE_NAME'] = name

### Institution

In [8]:
mummies_df.Institution.nunique()

260

In [9]:
institutions_dict = {
    '': '',
    ' Fondazione Museo delle Antichità Egizie': 'Egyptian Museum, Turin Italy',
    '(?) Museum of Fine Arts Houston': 'Museum of Fine Arts Houston',
    '(Tomb 1 & 2)': 'UNKNOWN',
    '08-480.2a-c': 'Brooklyn Museum',
    '11.6': 'Brooklyn Museum',
    '52.128a-e': 'Brooklyn Museum',
    '8898*88**': 'Garrett Evangelical Theological Seminary (Seabury-Western), Illinois',
    'Academy of Natural Sciences of Drexel University': 'Academy of Natural Sciences of Drexel University',
    'Aksaray Müzesi': 'Aksaray Museum',
    'Albany Institute of History & Art': 'Albany Institute of History and Art',
    'Albert Hall Museum': 'Albert Hall Museum',
    'Amasya Museum Hall of Mummies': 'Amasya Museum',
    'Archaeological Museum in Zagreb': 'Archaeological Museum in Zagreb',
    'Archaeological Museum of Zanjan': 'Archaeological Museum of Zanjan',
    'Assaraya Alhamra Museum': 'Assaraya Alhamra Museum',
    'Australia Museum': 'Australian Museum',
    'Australian Museum': 'Australian Museum',
    'Barnum Museum': 'Barnum Museum',
    'Basel University': 'Basel University',
    'Bavarian State Archaeological Collection': 'Bavarian State Archaeological Collection',
    'Berkshire Museum': 'Berkshire Museum',
    'Bolton Museum': 'Bolton Museum',
    'Boston Museum of Fine Arts': 'Museum of Fine Arts Boston',
    'British Museum': 'British Museum',
    'Brooklyn Museum': 'Brooklyn Museum',
    'Buffalo Museum of Science': 'Buffalo Museum of Science',
    'Burke Museum': 'Burke Museum',
    'CG24100': 'Egyptian Museum',
    'CG33281': 'Egyptian Museum',
    'Cairo Museum The Virtual Eqyptian Museum': 'Egyptian Museum',
    'Caldwell Heritage Museum': 'Caldwell Heritage Museum',
    'Canadian Museum of Civilization': 'Canadian Museum of Civilization',
    'Canterbury Museum, NZ': 'Canterbury Museum, NZ',
    'Carnegie Museum': 'Carnegie Museum',
    'Centre for the Study of the Arctic;Shemanovsky Science Museum and Exhibition Centre': 'Shemanovsky Science Museum and Exhibition Centre',
    'Chatham-Kent Museum': 'Chatham-Kent Museum',
    'Church of Kampehl': 'Church of Kampehl',
    'Church of San Frediano': 'Church of San Frediano',
    'Cincinnati Museum Center': 'Cincinnati Museum Center',
    'Civic Museum of Ligurian Archaeology, Genoa': 'Civic Museum of Ligurian Archaeology, Genoa',
    'Complexual National Muzeal ASTRA': 'Complexual National Muzeal ASTRA',
    'Corpus Domini Bologna': 'Corpus Domini Bologna',
    'Denvery Museum of Nature and Science': 'Denver Museum of Nature and Science',
    'Department of Egyptian Antiquities of the British Museum': 'British Museum',
    'Derby Museum': 'Derby Museum',
    'Detroit Institute of Art': 'Detroit Institute of Art',
    'Drents Museum': 'Drents Museum',
    'Druban Museum': 'Durban Museum',
    'EA21809': 'British Museum',
    'EA21811': 'British Museum',
    'EA6713': 'British Museum',
    'Ecuadorian Museum': 'Ecuadorian Museum',
    'Egypt Centre - Canolfan Eiffraidd, Swansea': 'Museum of Egyptian Antiquities, Swansea University',
    'Egyptian Museum': 'Egyptian Museum',
    'Egyptian Museum ': 'Egyptian Museum',
    'Egyptian Museum of Antiquities': 'Egyptian Museum',
    'Egyptian Museum, Torino Italy': 'Egyptian Museum, Turin Italy',
    'Egyptian Museum, Turin Italy': 'Egyptian Museum, Turin Italy',
    'Egyptian National Museum of Antiquities, Cairo': 'Egyptian Museum',
    'Eternal Egypt': 'British Museum',
    'Eve-Maria Zimmermann Collection, Tenerife REM, Mannheim Germany': 'Eve-Maria Zimmermann Collection',
    'Field Museum of National History, Chicago': 'Field Museum (Chicago)',
    'Fleming Museum': 'Robert Hull Fleming Museum',
    'Fondazione Museo delle Antichità Egizie': 'Egyptian Museum, Turin Italy',
    'Garstang Museum of Archaeology Liverpool': 'Garstang Museum of Archaeology Liverpool',
    'Geophysical Institute University of Alaska Fairbanks': 'Geophysical Institute University of Alaska Fairbanks',
    'Grand Rapids Public Museum': 'Grand Rapids Public Museum',
    'Hancock Museum': 'Grand Rapids Public Museum',
    'Haslemere Museum': 'Grand Rapids Public Museum',
    'Historisches Museum Bern CH': 'Museum of History and Ethnography, St. Gallen, Switzerland',
    'Historisches Museum urg, CHAargau, Schloss Lenz': 'Museum of History and Ethnography, St. Gallen, Switzerland',
    'Historisches und Volkerkundemuseum': 'Museum of History and Ethnography, St. Gallen, Switzerland',
    'Historisches und Volkerkundemuseum, St. Gallen CH': 'Museum of History and Ethnography, St. Gallen, Switzerland',
    'Huando Museum': 'Huando Museum',
    'Hunan Provincial Museum': 'Hunan Provincial Museum',
    'Hungarian National History Museum': 'Hungarian National History Museum',
    'Israel Museum': 'Israel Museum',
    'Iziko Museum': 'Iziko Museum',
    'J. Paul Getty Museum': 'J. Paul Getty Museum',
    'JB Getty Museum': 'J. Paul Getty Museum',
    'JE26206(b)': 'Egyptian Museum',
    'JE26216(b)': 'Egyptian Museum',
    'JE26218(b)+B47': 'Egyptian Museum',
    'JE26225(b)': 'Egyptian Museum',
    'JE56265(b)': 'Egyptian Museum',
    'JE59130(b)': 'Egyptian Museum',
    'Joseon Dynasty Human Sample Collection of Seol National University College of Medicine': 'Seoul National University',
    'KNH Centre for Biomedical Egyptology': 'KNH Centre for Biomedical Egyptology',
    'Kalamazoo Valley Museum': 'Kalamazoo Valley Museum',
    'Karaman Museum, Turkey': 'Karaman Museum, Turkey',
    'Kemper Art Museum': 'Kemper Art Museum',
    'Kingdom of Permafrost on Nov 17/15': 'Kingdom of Permafrost on Nov 17/15',
    'Kulturhistorisk Museum Oslo': 'Kulturhistorisk Museum Oslo',
    'Kunsthistorishces Museum': 'Kunsthistorisches Museum',
    'L33-002-006': 'Museum of World Treasures',
    'L53-001-006': 'Museum of World Treasures',
    'Leeds Museum': 'Leeds Museum',
    'Leicester Museum New Walk Museum & Art Gallery': 'Leicester Museum New Walk Museum & Art Gallery',
    'Leiden Museum': 'Leiden Museum',
    'Lippisches Landes Museum': 'Lippisches Landes Museum',
    'Liverpool Museums': 'Liverpool Museums',
    'Louvre, Paris': 'Louvre, Paris',
    'MMA 11.139': 'Metropolitan Museum of Art',
    'MMA 11.150.15c': 'Metropolitan Museum of Art',
    'MMA 12.182.131c': 'Metropolitan Museum of Art',
    'MMA 12.182.48c': 'Metropolitan Museum of Art',
    'MMA 25.3.219': 'Metropolitan Museum of Art',
    'MMA 26.3.12': 'Metropolitan Museum of Art',
    'MMA 86.1.35': 'Metropolitan Museum of Art',
    'MMA 86.1.51': 'Metropolitan Museum of Art',
    'MMA 99.3.5': 'Metropolitan Museum of Art',
    'Mabee-Gerrer Museum of Art': 'Mabee-Gerrer Museum of Art',
    'Maidstone Museum': 'Maidstone Museum, England',
    'Maidstone Museum, England': 'Maidstone Museum, England',
    'Manchester Museum': 'Manchester Museum',
    'Marischal Museum': 'Marischal Museum',
    'McClung Museum courtesy of Peabody Museum of Archaeology and Ethnology': 'McClung Museum',
    'McClung Museum purchased from Toledo Museum of Art 1991.7.144': 'McClung Museum',
    'McGill University': 'McGill University',
    'Medelhavsmuseet': 'Medelhavsmuseet',
    'Michael C. Carlos Museum': 'Michael C. Carlos Museum',
    'Milwaukee Public Museum': 'Milwaukee Public Museum',
    'Minneapolis Institute of Art': 'Minneapolis Institute of Art',
    'Monastery of Attel, Upper Bavaria': 'Monastery of Attel, Upper Bavaria',
    'Musee Royaux, Brussels': 'Musee Royaux, Brussels',
    "Musee d'Ethnographie": "Musee d'Ethnographie",
    "Musee d'Orbe, Switzerland": "Musee d'Orbe, Switzerland",
    "Musee d'Yverdon et region, Yverdon CH": "Museum Yverdon und Region, Yverdon-les-bains",
    'Musee de la Civilisation': 'Musee de la Civilisation',
    'Musei Vaticani': 'Musei Vaticani',
    'Musem of Fine Arts Boston': 'Museum of Fine Arts Boston',
    'Museo Nacional de Antropologia, Biodiversidad Agricultura y Alimentacion (MUNABA)': 'Museo Nacional de Antropologia, Biodiversidad Agricultura y Alimentacion (MUNABA)',
    'Museo Nacional, Lima': 'Museo Nacional, Lima',
    'Museo Santuarios de Altura': 'Museo Santuarios de Altura',
    'Museo de El Carmen': 'Museo de El Carmen',
    'Museo de la Memoria los Derechos Humanos': 'Museo de la Memoria los Derechos Humanos',
    'Museo di Anatomia Umana, Universita di Pisa': 'Museo di Anatomia Umana, Universita di Pisa',
    'Museum Victoria': 'Museum Victoria',
    'Museum Yverdon und Region, Yverdon-les-bains': 'Museum Yverdon und Region, Yverdon-les-bains',
    'Museum of Egyptian Antiquities, Swansea University': 'Museum of Egyptian Antiquities, Swansea University',
    'Museum of Fine Arts Boston': 'Museum of Fine Arts Boston',
    'Museum of History and Ethnography, St. Gallen, Switzerland': 'Museum of History and Ethnography, St. Gallen, Switzerland',
    'Museum of Man San Diego': 'Museum of Man San Diego',
    'Museum of Meditarranean and Near Eastern Antiquities (Stockholm)': 'Medelhavsmuseet',
    'Museum of Science': 'Museum of Science',
    'Museum of Taizhou': 'Museum of Taizhou',
    'Museum of Witchcraft': 'Museum of Witchcraft',
    'Museum of World Treasures': 'Museum of World Treasures',
    'Mutter Museum': 'Mutter Museum',
    'Myers Museum, Eton College': 'Myers Museum, Eton College',
    'National Cultural History Museum': 'National Cultural History Museum',
    'National Museum of Antiquities': 'National Museum of Antiquities',
    'National Museum of Archaeology, Lisbon Portugal': 'National Museum of Archaeology, Lisbon Portugal',
    'National Museum of Ireland': 'National Museum of Ireland',
    'National Museum of Natural History (Smithsonian Institution; USNM),': 'National Museum of Natural History',
    'National Museum of Natural History Department of Anthropology P381235': 'National Museum of Natural History',
    'National Museum of Scotland': 'National Museum of Scotland',
    'Nationalmuseet, Copenhagen DK': 'Nationalmuseet, Copenhagen DK',
    'Natural History Museum': 'Natural History Museum',
    'Natural History Museum of Los Angeles': 'Natural History Museum of Los Angeles',
    'Natural History Museum, Basel': 'Natural History Museum, Basel',
    'Natural History Museum, Vienna': 'Natural History Museum, Vienna',
    'Naturwissenschaftliche Sammlungen Winterthur': 'Naturwissenschaftliche Sammlungen Winterthur',
    'Nicholson Museum': 'Nicholson Museum University of Sydney',
    'Nicholson Museum University of Sydney': 'Nicholson Museum University of Sydney',
    'Norfolk Museums': 'Norfolk Museums',
    'Ohio History Connection': 'Ohio History Connection',
    'Old Capital Museum': 'Old Capitol Museum',
    'Otago Museum': 'Otago Museum',
    'PV.2010.44': 'UNKNOWN',
    'PV.2010.45': 'UNKNOWN',
    'Peabody Museum': 'Peabody Museum',
    'Peabody?': 'Peabody Museum',
    'Pechersk Lavra': 'Pechersk Lavra',
    'Penn Museum': 'University of Pennsylvania Museum',
    'Perth Museum and Art Gallery': 'Perth Museum and Art Gallery',
    'Queensland Museum': 'Queensland Museum',
    'REM, Mannheim, Germany': 'Reiss-Engelhorn-Museen, Mannheim DE',
    'RISD Museum': 'RISD Museum',
    'Ratisches Museum Chur CH': 'Ratisches Museum Chur CH',
    'Raymond Dart Collection': 'Raymond Dart Collection',
    'Redpath Museum': 'Redpath Museum',
    'Reiss-Engelhorn-Museen, Mannheim DE': 'Reiss-Engelhorn-Museen, Mannheim DE',
    'Rijksmuseum van Oudheden': 'National Museum of Antiquities',
    'Robert Hull Fleming Museum': 'Robert Hull Fleming Museum',
    'Rosicrucian Egyptian Museum': 'Rosicrucian Egyptian Museum',
    'Royal Ontario Museum': 'Royal Ontario Museum',
    'Rueil-Malmaison museum': 'Rueil-Malmaison museum',
    'Rutgers Geology Museum on loan from New Brunswick Theological Seminary': 'Rutgers Geology Museum',
    'Ryusui-ji Dainchibou Temple': 'Ryusui-ji Dainchibou Temple',
    'SR4.11326(b)': 'UNKNOWN',
    'Saint Louise Art Museum': 'Saint Louis Art Museum',
    'San Francisco Fine Arts Museum': 'San Francisco Fine Arts Museum',
    'San Fransico State University Museum': 'San Francisco State University Museum',
    'Schloss Gottorf, Schleswig DE': 'Schloss Gottorf, Schleswig DE',
    'Shemanovsky Museum': 'Shemanovsky Science Museum and Exhibition Centre',
    'Shrewsbury Museum & Art Gallery': 'Shrewsbury Museum & Art Gallery',
    'Society of Vertebrate Paleontology': 'Society of Vertebrate Paleontology',
    'South Australia Museum': 'South Australian Museum',
    'South Australian Museum': 'South Australian Museum',
    'South Tyrol Museum of Archaeology': 'South Tyrol Museum of Archaeology',
    'Southern Baptist Theological Seminary': 'Southern Baptist Theological Seminary',
    'Speed Art Museum': 'Speed Art Museum',
    'Spurlock Museum University of Illinois': 'Spurlock Museum University of Illinois',
    "St. Michan's Church": "St. Michan's Church",
    'St. Petersburg Museum of History': 'St. Petersburg Museum of History',
    'Staatliche Kunstsammlungen Dresed': 'Staatliche Kunstsammlungen Dresden',
    'State Archaeology Museum (India)': 'State Archaeology Museum (India)',
    'State Museum, India': 'State Archaeology Museum (India)',
    'Stiftsbibliothe, St. Gallen': 'Stiftsbibliothek, St. Gallen',
    'Swansea Museum U.K.': 'Museum of Egyptian Antiquities, Swansea University',
    'TR13.3.15.1(d)': 'UNKNOWN',
    'TR13.3.15.1(f)': 'UNKNOWN',
    'TR13.3.15.1€': 'UNKNOWN',
    'Tasmanian Museum & Art Gallery': 'Tasmanian Museum & Art Gallery',
    'Te Papa (prev. Colonial Museum)': 'Te Papa (prev. Colonial Museum)',
    'Te Papa Museum': 'Te Papa (prev. Colonial Museum)',
    'The Art Institute of Chicago': 'The Art Institute of Chicago',
    'The British Museum': 'British Museum',
    'The Field Museum': 'Field Museum (Chicago)',
    'The Field Museum (Chicago)': 'Field Museum (Chicago)',
    'The Fitzwilliam Museum': 'Fitzwilliam Museum',
    'The Global Egyptian Museum/Kunsthistorisches Museum': 'Kunsthistorisches Museum',
    'The Kelsey Museum of Archaeology': 'Kelsey Museum of Archaeology',
    'The Metropolitan Museum of Art': 'Metropolitan Museum of Art',
    'The Oriental Insitute University of Chicago': 'Oriental Institute University of Chicago',
    'The Walters Art Museum': 'Walters Art Museum',
    'The Walters Art Museum on loan from Goucher College, Baltimore': 'Walters Art Museum',
    'Tokyo National Museum': 'Tokyo National Museum',
    'Torquay Museum': 'Torquay Museum',
    'Tulane': 'Tulane',
    'UPM L-55-15A': 'University of Pennsylvania Museum',
    'UiO Museum of Cultural Histor': 'Museum of Cultural History, Oslo',
    'UiO Museum of Cultural History': 'Museum of Cultural History, Oslo',
    'Ulster Museum': 'Ulster Museum',
    'Universalmuseeum Joanneum Graz AU': 'Universalmuseum Joanneum Graz AU',
    'Universite de Quebec a Montreal': 'Universite de Quebec a Montreal',
    'University Museums of Padua': 'University Museums of Padua',
    'University of Alaska Musem of the North': 'University of Alaska Museum of the North',
    'University of Pennsylvania Museum of Archaeology and Anthropology (UPM),': 'University of Pennsylvania Museum',
    'University of Pisa': 'University of Pisa',
    'University of Richmond': 'University of Richmond',
    'University of Sydney': 'University of Sydney',
    'University of Western Ontario': 'University of Western Ontario',
    'University of Wyoming': 'University of Wyoming',
    'Vatican Museum': 'Musei Vaticani',
    'Victor Balaguer Library Museum': 'Victor Balaguer Library Museum',
    'Virginia Museum of Fine Arts': 'Virginia Museum of Fine Arts',
    'Volkerkundemuseum Basel CH': 'Volkerkundemuseum Basel CH',
    'Von Luschan & American Museum of Natural History (AMNH)': 'American Museum of Natural History',
    'Wayne County Historical Museum': 'Wayne County Historical Museum',
    'Westminster College': 'Westminster College',
    'Williams College Musem of Art': 'Williams College Musem of Art',
    'World Museum Liverpool': 'World Museum Liverpool',
    'Xingjiang Museum': 'Xinjiang Museum',
    'Yale Peabody': 'Yale Peabody',
    'de Young Museum': 'de Young Museum',
    'skiff': 'UNKNOWN',
    'wnichols@museumofvancouver.ca': 'UNKNOWN',
}

In [10]:
institutions_locs = { # lat, lng
    '': 'UNKNOWN',
    'Academy of Natural Sciences of Drexel University': (39.957, -75.1714),
    'Aksaray Museum': (38.361111, 33.993611),
    'Albany Institute of History and Art': (42.655774, -73.760372),
    'Albert Hall Museum': (26.91179, 75.81953),
    'Amasya Museum': (40.6483985,35.8227312),
    'American Museum of Natural History': (40.780556, -73.974722),
    'Archaeological Museum in Zagreb': (45.811111, 15.977222),
    'Archaeological Museum of Zanjan': (36.6709199,48.4784561),
    'Assaraya Alhamra Museum': (32.895833, 13.180278),
    'Australian Museum': (-33.877432, 151.212732),
    'Barnum Museum': (41.175278, -73.188611),
    'Basel University': (47.55858, 7.5836),
    'Bavarian State Archaeological Collection': (48.143889, 11.591111),
    'Berkshire Museum': (42.4473, -73.2536),
    'Bolton Museum': (53.5774, -2.431557),
    'British Museum': (51.519459, -0.126931),
    'Brooklyn Museum': (40.671306, -73.96375),
    'Buffalo Museum of Science': (42.906132, -78.843314),
    'Burke Museum': (47.660972, -122.309778),
    'Caldwell Heritage Museum': (35.9116091,-81.5385798),
    'Canadian Museum of Civilization': (45.429722, -75.708889),
    'Canterbury Museum, NZ': (-43.5309, 172.6271),
    'Carnegie Museum': (40.44369, -79.948976),
    'Shemanovsky Science Museum and Exhibition Centre': (66.5359062,66.607585),
    'Chatham-Kent Museum': (42.3891543,-82.151762),
    'Church of Kampehl': (52.866667, 12.433056),
    'Church of San Frediano': (43.846111, 10.504722),
    'Cincinnati Museum Center': (39.11, -84.537778),
    'Civic Museum of Ligurian Archaeology, Genoa': (44.4291656,8.8126433),
    'Complexual National Muzeal ASTRA': (45.7975571,24.1510981),
    'Corpus Domini Bologna': (44.489167, 11.339722),
    'Denver Museum of Nature and Science': (39.7475, -104.9425),
    'Derby Museum': (38.2053, -85.7712),
    'Detroit Institute of Art': (42.359292, -83.064797),
    'Drents Museum': (52.9933, 6.5644),
    'Durban Museum': (-29.883333, 31.05),
    'Ecuadorian Museum': (-0.233333, -78.516667),
    'Egyptian Museum': (30.047778, 31.233333),
    'Egyptian Museum, Turin Italy': (45.068333, 7.684444),
    'Eve-Maria Zimmermann Collection': 'UNKNOWN',
    'Field Museum (Chicago)': (41.86611, -87.61694),
    'Fitzwilliam Museum': (52.200278, 0.119444),
    'Garrett Evangelical Theological Seminary (Seabury-Western), Illinois': (42.0562, -87.6755),
    'Garstang Museum of Archaeology Liverpool': (53.4022133,-2.9644639),
    'Geophysical Institute University of Alaska Fairbanks': (64.859444, -147.849722),
    'Grand Rapids Public Museum': (42.965528,-85.676715),
    'Huando Museum': (-11.486175,-77.1836962),
    'Hunan Provincial Museum': (28.2153, 112.9878),
    'Hungarian National History Museum': (47.482222, 19.085556),
    'Israel Museum': (31.772379, 35.204524),
    'Iziko Museum': (-33.928889, 18.415),
    'J. Paul Getty Museum': (34.0775, -118.475),
    'Seoul National University': (37.46, 126.95),
    'KNH Centre for Biomedical Egyptology': (53.4668498,-2.2360777),
    'Kalamazoo Valley Museum': (42.29329, -85.58392),
    'Karaman Museum, Turkey': (37.182778, 32.211389),
    'Kelsey Museum of Archaeology': (42.276861, -83.740556),
    'Kemper Art Museum': (38.6471177,-90.3027034),
    'Kingdom of Permafrost on Nov 17/15': 'UNKNOWN',
    'Kulturhistorisk Museum Oslo': (59.9167934,10.735523),
    'Kunsthistorisches Museum': (48.2037, 16.3614),
    'Leeds Museum': (53.801617, -1.547323),
    'Leicester Museum New Walk Museum & Art Gallery': (52.628954, -1.127765),
    'Leiden Museum': (52.1270298,3.0515736),
    'Lippisches Landes Museum': (51.93715,8.874576),
    'Liverpool Museums': (53.4185418,-3.0250408),
    'Louvre, Paris': (48.8606111,2.337644),
    'Mabee-Gerrer Museum of Art': (35.366848, -96.953704),
    'Maidstone Museum, England': (51.27602,0.520466),
    'Manchester Museum': (53.4766765,-2.2954527),
    'Marischal Museum': (57.1489883,-2.0993574),
    'McClung Museum': (35.951999, -83.927178),
    'McGill University': (45.504167, -73.574722),
    'Medelhavsmuseet': (59.279511, 18.067467),
    'Metropolitan Museum of Art': (40.77891, -73.96367),
    'Michael C. Carlos Museum': (33.790278, -84.324167),
    'Milwaukee Public Museum': (43.040744, -87.921095),
    'Minneapolis Institute of Art': (44.958611, -93.274167),
    'Monastery of Attel, Upper Bavaria': (47.5698601,11.0951279),
    'Musee Royaux, Brussels': (50.8419778,4.3579037),
    "Musee d'Ethnographie": (46.197946, 6.136959),
    "Musee d'Orbe, Switzerland": (46.7233168,6.5325746),
    'Musee de la Civilisation': (46.815108, -71.202383),
    'Musei Vaticani': (41.906389, 12.454444),
    'Museo Nacional de Antropologia, Biodiversidad Agricultura y Alimentacion (MUNABA)': (-12.0737506,-77.0392978),
    'Museo Nacional, Lima': (-12.077222, -77.061944),
    'Museo Santuarios de Altura': (-16.3995748,-71.5376988),
    'Museo de El Carmen': (19.345, -99.1894),
    'Museo de la Memoria los Derechos Humanos': (33.439839, -70.679375),
    'Museo di Anatomia Umana, Universita di Pisa': (43.714596,10.3947562),
    'Museum Victoria': (-37.8032651,144.9717239),
    'Museum Yverdon und Region, Yverdon-les-bains': (46.7783713,6.6416352),
    'Museum of Cultural History, Oslo': (59.916806, 10.735472),
    'Museum of Egyptian Antiquities, Swansea University': (51.6101373,-3.9788101),
    'Museum of Fine Arts Boston': (42.339167, -71.094167),
    'Museum of Fine Arts Houston': (29.725694, -95.390417),
    'Museum of History and Ethnography, St. Gallen, Switzerland': (47.4286119,9.3807913),
    'Museum of Man San Diego': (32.731757, -117.152866),
    'Museum of Science': (51.4975, -0.174722),
    'Museum of Taizhou': (28.666667, 121.35),
    'Museum of Witchcraft': (50.6906674,-4.6949164),
    'Museum of World Treasures': (37.687305, -97.32789),
    'Mutter Museum': (39.953333, -75.176667),
    'Myers Museum, Eton College': (51.4957355,-0.6066283),
    'National Cultural History Museum': (-25.7532287,28.1847306),
    'National Museum of Antiquities': (52.158333, 4.485833),
    'National Museum of Archaeology, Lisbon Portugal': (38.697278, -9.207056),
    'National Museum of Ireland': (53.5905411,-9.9731335),
    'National Museum of Natural History': (38.8913, -77.0259),
    'National Museum of Scotland': (55.946944, -3.19),
    'Nationalmuseet, Copenhagen DK': (55.674722, 12.574722),
    'Natural History Museum': (51.495983, -0.176372),
    'Natural History Museum of Los Angeles': (34.016989, -118.288781),
    'Natural History Museum, Basel': (47.5575, 7.590556),
    'Natural History Museum, Vienna': (48.2052, 16.3599),
    'Naturwissenschaftliche Sammlungen Winterthur': (47.5018,8.73001),
    'Nicholson Museum University of Sydney': (-33.8864, 151.1888),
    'Norfolk Museums': (52.6645501,0.5048529),
    'Ohio History Connection': (39.983333, -82.983333),
    'Old Capitol Museum': (41.661344, -91.535647),
    'Oriental Institute University of Chicago': (41.789291, -87.597482),
    'Otago Museum': (-45.865678, 170.510842),
    'Peabody Museum': (42.378194, -71.114806),
    'Pechersk Lavra': (50.434167, 30.559167),
    'Perth Museum and Art Gallery': (56.3983, -3.4283),
    'Queensland Museum': (-23.3635596,145.4515795),
    'RISD Museum': (41.82635,-71.4099105),
    'Ratisches Museum Chur CH': (46.848163,9.5335738),
    'Raymond Dart Collection': (-26.190833, 28.030278),
    'Redpath Museum': (45.5044, -73.5776),
    'Reiss-Engelhorn-Museen, Mannheim DE': (49.48888, 8.46212),
    'Robert Hull Fleming Museum': (44.48059, -73.199),
    'Rosicrucian Egyptian Museum': (37.3342, -121.9233),
    'Royal Ontario Museum': (43.667679, -79.394809),
    'Rueil-Malmaison museum': (48.876, 2.181),
    'Rutgers Geology Museum': (40.4986387,-74.4488605),
    'Ryusui-ji Dainchibou Temple': (38.5925224,139.8981153),
    'Saint Louis Art Museum': (38.639444, -90.294444),
    'San Francisco Fine Arts Museum': (37.771516,-122.468647),
    'San Francisco State University Museum': (37.7220789,-122.4814187),
    'Schloss Gottorf, Schleswig DE': (54.511667, 9.541389),
    'Shrewsbury Museum & Art Gallery': (52.70715, -2.754811),
    'Society of Vertebrate Paleontology': (39.014049,-77.1028812),
    'South Australian Museum': (-34.920783, 138.603017),
    'South Tyrol Museum of Archaeology': (46.499869, 11.349531),
    'Southern Baptist Theological Seminary': (38.24846, -85.68689),
    'Speed Art Museum': (38.217861, -85.760917),
    'Spurlock Museum University of Illinois': (40.1073, -88.2214),
    "St. Michan's Church": (53.347455, -6.275547),
    'St. Petersburg Museum of History': (27.7738, -82.6305),
    'Staatliche Kunstsammlungen Dresden': (51.052778, 13.736944),
    'State Archaeology Museum (India)': (17.3992207,78.4680745),
    'Stiftsbibliothek, St. Gallen': (47.4228, 9.3764),
    'Tasmanian Museum & Art Gallery': (-42.8819, 147.3319),
    'Te Papa (prev. Colonial Museum)': (-41.2904563,174.7799007),
    'The Art Institute of Chicago': (41.879444, -87.623889),
    'Tokyo National Museum': (35.718889, 139.775833),
    'Torquay Museum': (50.4624475,-3.5184133),
    'Tulane': (29.935344, -90.122687),
    'UNKNOWN': 'UNKNOWN',
    'Ulster Museum': (54.582, -5.935),
    'Universalmuseum Joanneum Graz AU': (47.0725, 15.436814),
    'Universite de Quebec a Montreal': (45.513889, -73.560278),
    'University Museums of Padua': (45.4042485,11.8656288),
    'University of Alaska Museum of the North': (64.8585, -147.8423),
    'University of Pennsylvania Museum': (39.9522188,-75.1932137),
    'University of Pisa': (43.716508, 10.398458),
    'University of Richmond': (37.57516, -77.53871),
    'University of Sydney': (-33.8777771,151.1821609),
    'University of Western Ontario': (43.008289, -81.271894),
    'University of Wyoming': (41.313056, -105.581389),
    'Victor Balaguer Library Museum': (41.224444, 1.725833),
    'Virginia Museum of Fine Arts': (37.55698, -77.47396),
    'Volkerkundemuseum Basel CH': (47.557222, 7.590833),
    'Walters Art Museum': (39.296667, -76.616111),
    'Wayne County Historical Museum': (39.8309514,-84.8891234),
    'Westminster College': (41.117778, -80.329444),
    'Williams College Musem of Art': (42.711186, -73.202692),
    'World Museum Liverpool': (53.409992, -2.981323),
    'Xinjiang Museum': (43.818, 87.58),
    'Yale Peabody': (41.317538, -72.919863),
    'de Young Museum': (37.771389, -122.468611),
}

## Processing data

In [11]:
g = nx.Graph()
for x in mummies_df.Institution:
    name = institutions_dict[str(x)]
    loc = institutions_locs[name]
    if loc != 'UNKNOWN':
        if not g.has_node(name):
            g.add_node(name, lat=loc[0], lng=loc[1], weight=0)
        g.node[name]['weight'] += 1

## Saving data

In [12]:
nx.write_gexf(g, 'data/graph_mummies.gexf')